In [1]:
import xarray as xr
import cftime 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import os
import time
import logging
import cartopy.crs as ccrs
import metpy  
import calendar
import argparse
import glob

In [2]:
run = 'parallel_new_input'
os.chdir('/home/smmrrr/TEM/TEM_Runs/'+run)

print(os.getcwd()) 


/home/smmrrr/TEM/TEM_Runs/parallel_new_input


In [5]:
output_paths = glob.glob('*.csv*')
input_paths = glob.glob('*.HVD*')

print(len(output_paths))
print(len(input_paths))

500
250


In [25]:
print(len(output_paths))
print(len(output_group))
print(len(output_var))

500
500
500


In [22]:
np.unique(output_var)

array(['A', 'C', 'D', 'E', 'I', 'L', 'P', 'R', 'S', 'T', 'V'], dtype='<U1')

In [32]:

output_col_names = ['lon'
,'lat'
,'variable'
,'cohort_number'
,'stand_age'
,'potential_veg'
,'current_veg'
,'subtype'
,'community_type'
,'lc_state'
,'silt_clay'
,'QC_Value'
,'land_area'
,'cohort_area'
,'year'
,'annual_sum'
,'monthly_maximum'
,'monthly_mean'
,'monthly_minimum'
,'Jan'
,'Feb'
,'Mar'
,'Apr'
,'May'
,'Jun'
,'Jul'
,'Aug'
,'Sep'
,'Oct'
,'Nov'
,'Dec'
,'region']

input_col_names = ["lon", 'lat','var' ,'Area', 'year', 'sum', 'max', 'average'
         , 'min', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct'
         , 'Nov', 'Dec', 'Area_Name']

In [23]:

output_group = [re.findall('[0-9]+$', w) for w in output_paths]
output_group = [item for sublist in output_group for item in sublist]

output_var = [re.findall('AVLN|NCE|NPP|SOLC|VEGC|GPP|NEP|SMOIST|SOLN|VEGN', w) for w in output_paths]
output_var = [item for sublist in output_var for item in sublist]

input_group = [re.findall('[0-9]+$', w) for w in input_paths]
input_group = [item.upper() for sublist in input_group for item in sublist]

input_var = [re.findall('PREC|CLDS|PAR|TAIR|VPR', w) for w in input_paths]
input_var = [item for sublist in input_var for item in sublist]


array(['AVLN', 'GPP', 'NCE', 'NEP', 'NPP', 'SMOIST', 'SOLC', 'SOLN',
       'VEGC', 'VEGN'], dtype='<U6')

In [15]:
# output_group

In [26]:
#make tables with the relevant read in info

input_data = pd.DataFrame(
{'input_paths': input_paths,
'input_group': input_group,
 'input_var': input_var
}
)

output_data = pd.DataFrame(
{'output_paths': output_paths,
'output_group': output_group,
 'output_var': output_var
}
)

In [27]:
input_data

,input_paths,input_group,input_var
0,CLDS.HVD544,544,CLDS
1,PAR.HVD2244,2244,PAR
2,TAIR.HVD2720,2720,TAIR
3,PAR.HVD2040,2040,PAR
4,PREC.HVD952,952,PREC
...,...,...,...
245,VPR.HVD1360,1360,VPR
246,CLDS.HVD1700,1700,CLDS
247,TAIR.HVD1292,1292,TAIR
248,TAIR.HVD3196,3196,TAIR


In [28]:

input_data = pd.pivot(input_data, index = 'input_group', columns = 'input_var', values = 'input_paths').reset_index()

read_in_data = input_data.merge(output_data, left_on = 'input_group', right_on = 'output_group')

In [29]:
read_in_data

,input_group,CLDS,PAR,PREC,TAIR,VPR,output_paths,output_group,output_var
0,0,CLDS.HVD0,PAR.HVD0,PREC.HVD0,TAIR.HVD0,VPR.HVD0,SOLN_LULCC.csv0,0,SOLN
1,0,CLDS.HVD0,PAR.HVD0,PREC.HVD0,TAIR.HVD0,VPR.HVD0,VEGN_LULCC.csv0,0,VEGN
2,0,CLDS.HVD0,PAR.HVD0,PREC.HVD0,TAIR.HVD0,VPR.HVD0,NCE_LULCC.csv0,0,NCE
3,0,CLDS.HVD0,PAR.HVD0,PREC.HVD0,TAIR.HVD0,VPR.HVD0,SMOIST_LULCC.csv0,0,SMOIST
4,0,CLDS.HVD0,PAR.HVD0,PREC.HVD0,TAIR.HVD0,VPR.HVD0,AVLN_LULCC.csv0,0,AVLN
...,...,...,...,...,...,...,...,...,...
495,952,CLDS.HVD952,PAR.HVD952,PREC.HVD952,TAIR.HVD952,VPR.HVD952,AVLN_LULCC.csv952,952,AVLN
496,952,CLDS.HVD952,PAR.HVD952,PREC.HVD952,TAIR.HVD952,VPR.HVD952,NCE_LULCC.csv952,952,NCE
497,952,CLDS.HVD952,PAR.HVD952,PREC.HVD952,TAIR.HVD952,VPR.HVD952,SOLN_LULCC.csv952,952,SOLN
498,952,CLDS.HVD952,PAR.HVD952,PREC.HVD952,TAIR.HVD952,VPR.HVD952,NEP_LULCC.csv952,952,NEP


In [34]:
read_in_data = read_in_data.loc[read_in_data['output_var'] == 'NEP'].reset_index()

index                        7
input_group                  0
CLDS                 CLDS.HVD0
PAR                   PAR.HVD0
PREC                 PREC.HVD0
TAIR                 TAIR.HVD0
VPR                   VPR.HVD0
output_paths    NEP_LULCC.csv0
output_group                 0
output_var                 NEP
Name: 0, dtype: object

In [46]:
## TEM PFT info
# forest_vegs = [9, 4, 16, 33, 19, 10 ]
# forest_veg_names = ['needle-leaf evergreen temperate','boreal','broadleaved evergreen tree tropical'
#                    ,'broadleaved evergreen tree temperate','broadleaved deciduous tree tropical'
#                    ,'broadleaved deciduous tree temperate']


ouput_data_path = '/home/smmrrr/TEM_Analysis/test_parallel_run/'

# for i in range(len(read_in_data)):
for i in range(len(read_in_data)):

###read in cohort output
    cohort_output = pd.read_csv(read_in_data.loc[i, 'output_paths']
                , names = output_col_names)

    ##read in precipitation files
    prec = pd.read_csv(read_in_data.loc[i, 'PREC']
               , names = input_col_names)
    par = pd.read_csv(read_in_data.loc[i, 'PAR']
               , names = input_col_names)
    clds = pd.read_csv(read_in_data.loc[i, 'CLDS']
               , names = input_col_names)
    tair = pd.read_csv(read_in_data.loc[i, 'TAIR']
               , names = input_col_names)
    vpr = pd.read_csv(read_in_data.loc[i, 'VPR']
               , names = input_col_names)


    ## shape input datasets wide to long
    prec = pd.melt(prec, id_vars = ['lon','lat', 'Area', 'year'], value_vars = [
    'Jan',
    'Feb',
    'Mar',
    'Apr',
    'May',
    'Jun',
    'Jul',
    'Aug',
    'Sep',
    'Oct',
    'Nov',
    'Dec'], 
             var_name='month', value_name='prec'
            ,col_level=0)

    par = pd.melt(par, id_vars = ['lon','lat', 'Area', 'year'], value_vars = [
    'Jan',
    'Feb',
    'Mar',
    'Apr',
    'May',
    'Jun',
    'Jul',
    'Aug',
    'Sep',
    'Oct',
    'Nov',
    'Dec'], 
             var_name='month', value_name='par'
            ,col_level=0)

    clds = pd.melt(clds, id_vars = ['lon','lat', 'Area', 'year'], value_vars = [
    'Jan',
    'Feb',
    'Mar',
    'Apr',
    'May',
    'Jun',
    'Jul',
    'Aug',
    'Sep',
    'Oct',
    'Nov',
    'Dec'], 
             var_name='month', value_name='clds'
            ,col_level=0)

    tair = pd.melt(tair, id_vars = ['lon','lat', 'Area', 'year'], value_vars = [
    'Jan',
    'Feb',
    'Mar',
    'Apr',
    'May',
    'Jun',
    'Jul',
    'Aug',
    'Sep',
    'Oct',
    'Nov',
    'Dec'], 
             var_name='month', value_name='tair'
            ,col_level=0)
    
    vpr = pd.melt(vpr, id_vars = ['lon','lat', 'Area', 'year'], value_vars = [
    'Jan',
    'Feb',
    'Mar',
    'Apr',
    'May',
    'Jun',
    'Jul',
    'Aug',
    'Sep',
    'Oct',
    'Nov',
    'Dec'], 
             var_name='month', value_name='vpr'
            ,col_level=0)

    ##merge climate variables together
    climate_vars = prec.merge(
                par,on = ['lon','lat', 'Area', 'year', 'month']
                ).merge(
                tair ,on = ['lon','lat', 'Area', 'year', 'month']                           
                ).merge(
                clds ,on = ['lon','lat', 'Area', 'year', 'month']                           
                ).merge(
                vpr ,on = ['lon','lat', 'Area', 'year', 'month']                           
                )

    ##subset to forest veg types 
    # cohort_output = cohort_output.loc[cohort_output['current_veg'].isin(forest_vegs)] 

    ##melt cohort output
    cohort_output = pd.melt(cohort_output, id_vars = ['lon','lat', 'cohort_area', 'land_area', 'year', 'current_veg', 'stand_age'], value_vars = [
    'Jan',
    'Feb',
    'Mar',
    'Apr',
    'May',
    'Jun',
    'Jul',
    'Aug',
    'Sep',
    'Oct',
    'Nov',
    'Dec'], 
             var_name='month', value_name='NCE'
            ,col_level=0)


    agg_cohort_output = cohort_output.merge(
                climate_vars, how = 'left'
        ,on = ['lon','lat', 'year', 'month']
                )


    
    # agg_cohort_output.to_csv(ouput_data_path+'merged_dataset_'+read_in_data.loc[i, 'output_group']+'.csv', index=False)
    agg_cohort_output.count()

    cohort_area = lambda x: np.average(x, weights=agg_cohort_output.loc[x.index, "cohort_area"])  # https://stackoverflow.com/questions/31521027/groupby-weighted-average-and-sum-in-pandas-dataframe
    land_area = lambda x: np.average(x, weights=agg_cohort_output.loc[x.index, "land_area"])  # https://stackoverflow.com/questions/31521027/groupby-weighted-average-and-sum-in-pandas-dataframe

    if (i==0):


        merged_dataset_annual_loop = agg_cohort_output.groupby([
                'month','year','current_veg', 'stand_age']).agg(cohort_area=('cohort_area', 'sum'), total_area=('land_area', 'sum'),
                                                                                      NCE_weighted=('NCE', cohort_area), 
                                                                                   prec = ('prec', land_area),
                                                                                   par = ('par', land_area),
                                                                                   tair = ('tair', land_area),
                                                                                   clds = ('clds', land_area),
                                                                                   vpr = ('vpr', land_area))
                                                                                    
    else:
                                                        
        merged_dataset_annual_loop = pd.concat([merged_dataset_annual_loop,
                              agg_cohort_output.groupby([
                'month','year','current_veg', 'stand_age']).agg(cohort_area=('cohort_area', 'sum'), total_area=('land_area', 'sum'),
                                                                                      NCE_weighted=('NCE', cohort_area), 
                                                                                   prec = ('prec', land_area),
                                                                                   par = ('par', land_area),
                                                                                   tair = ('tair', land_area),
                                                                                   clds = ('clds', land_area),
                                                                                   vpr = ('vpr', land_area))])
   
    merged_dataset_annual_loop.to_csv(ouput_data_path+'merged_dataset_annual_loop.csv')
    print(i)
    print(merged_dataset_annual_loop.count())


/home/smmrrr/miniconda3/envs/condaforge/lib/python3.10/site-packages/pandas/core/reshape/merge.py:1207: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation.
  warnings.warn(
/tmp/ipykernel_54929/1313626318.py:159: FutureWarning: Dropping invalid columns in SeriesGroupBy.agg is deprecated. In a future version, a TypeError will be raised. Before calling .agg, select only columns which should be valid for the function.
  merged_dataset_annual_loop = agg_cohort_output.groupby([


TypeError: unsupported operand type(s) for /: 'str' and 'int'

In [ ]:
cohort_output.count()